In [62]:
import numpy as np
import pandas as pd
import math as m
import sympy as sym
import random as rand
import matplotlib.pyplot as plt
import cmath as cm

In [63]:
# defininf the states
# defininsg coefficeients sybols

def Creating_states( coeff = [1/(2**.5),1/(2**.5),1,0] , Abstract = False):    # coeff list like [a0,a1,b0,b1]
    if Abstract == True:
        a0 = sym.symbols('a0')
        a1 = sym.symbols('a1')
        b0 = sym.symbols('b0')
        b1 = sym.symbols('b1')
    else:
        a0 = coeff[0]
        a1 = coeff[1]
        b0 = coeff[2]
        b1 = coeff[3]
    psi0 = [a0,a1]      # defining states
    psi1 = [b0,b1]
    return([psi0,psi1])

In [64]:
psi0 = Creating_states(Abstract=False)[0]
psi1 = Creating_states(Abstract=False)[1]

psi0

[0.7071067811865475, 0.7071067811865475]

In [65]:
psi0sq = []
psi1sq = []
[[psi0sq.append(i*j) for i in psi0] for j in psi0]
[[psi1sq.append(i*j) for i in psi1] for j in psi1]   # creating list of coeff for squared states (intermeditate step)

psi0psi0 = [psi0sq[0], (psi0sq[1]**2+psi0sq[2]**2)**(.5), psi0sq[3]]        # the second element is the normalized coefficient due to the addition of states (01+10). See OneNote writeup for details
psi1psi1 = [psi1sq[0], (psi1sq[1]**(2)+psi1sq[2]**(2))**(.5), psi1sq[3]]            # symmetric states in basis (|00>, normalized version of (|01>+|10>),|11>)

# psi0psi0
# sum([i**2 for i in psi0psi0])   # adding to 1 now.

In [66]:
#creating the density matrix rho
# rho is 1/2*(|psi0psi0Xpsi0psi0|+|psi1psi1Xpsi1psi1|)
# findiing the conjugate vectors 
vec_psi0psi0 = sym.Matrix(psi0psi0)     # convertying to sympy vectors for easy matrix multiplication
vec_psi1psi1 = sym.Matrix(psi1psi1)

rho = (1/2)*(vec_psi0psi0*vec_psi0psi0.T  + vec_psi1psi1*vec_psi1psi1.T)       # density matrix with priors 1/2 each.
rho

Matrix([
[            0.625, 0.176776695296637,             0.125],
[0.176776695296637,              0.25, 0.176776695296637],
[            0.125, 0.176776695296637,             0.125]])

In [67]:
# creating the SIC POVM matrices
w = m.e**((2/3)*m.pi*(1j))
POVM_vec = (1/(2**.5))*(np.array([[0,1,-1],[-1,0,1],[1,-1,0],[0,w,-w**2],[-1,0,w**2],[1,-w,0],[0,w**2,-w],[-1,0,w],[1,-w**2,0]]))
POVM_elts = [(1/3)*np.outer(np.conjugate(POVM_vec[i]),POVM_vec[i]) for i in range(len(POVM_vec))]
# sum([np.trace(POVM_elts[i]) for i in range(9)])`# trace sum checks out`

In [68]:
# prob_vec = [(POVM_elts[i]*rho).trace() for i in range(len(POVM_elts))]
prob_vec =  [np.trace(np.dot(POVM_elts[i],rho)) for i in range(9)] #[(POVM_elts[i]*rho).trace() for i in range(len(POVM_elts))]
# prob_vec
sum(prob_vec)        # prob not adding to 1?
# print(prob_vec)


0.999999999999999

In [69]:
# # verifying that the trace still holds to be the same, equalt to .111 for this weird POVM element.
# c = .00308641975308642*2.71828182845905**(4.18879020478639*(sym.I))

# p = cm.polar(c)
# cm.rect(p[0], p[1])

# r = .00617-.0015
# i = .0026

# ab = (r**2+i**2)**(.5)
# 2*ab # verified

In [70]:
N = 1000000       # number of measurements  

POVM_dir_symbols = ['d1','d2','d3','d4','d5','d6','d7','d8','d9']       # symbols to indicate collapsed direction
#prob distribution is simply the corresponding elements of the prob_vec
collapse_dir_vec = rand.choices(POVM_dir_symbols, weights=prob_vec, k = N)   # choosing collapse directions with weights for N trials

nj_vec = [collapse_dir_vec.count(f'd{i+1}') for i in range(9)]         # working babieee
pj_num_vec = [i/N for i in nj_vec]         # working babieee
([i*N for i in prob_vec] , nj_vec)   # comparing the theoretical and experimental collapse prob.

([3574.43490112103,
  83333.3333333333,
  86907.7682344544,
  91962.7825494394,
  145833.333333333,
  175296.115882773,
  91962.7825494394,
  145833.333333333,
  175296.115882773],
 [3550, 83310, 86844, 91999, 145996, 175761, 91690, 145948, 174902])

In [71]:
M = [[np.trace(np.dot(POVM_elts[i],POVM_elts[j])) for i in range(len(POVM_elts))] for j in range(len(POVM_elts))]     # creating M matrix using POVM definition


In [72]:
u_0 = [1/3 for i in range(9)]           # cerating u_0 vector, to create the inverse matrix
M_inv = 3*np.outer(u_0,u_0) + 12*(np.eye(9) - np.outer(u_0,u_0))        # creating the inverse matrix
# np.trace(np.dot(M,M_inv))       # gives identity, checks out
r_vec = np.dot(M_inv,pj_num_vec)


In [73]:
rho_num_list = [r_vec[i]*POVM_elts[i] for i in range(len(POVM_elts))]
[i for i in rho_num_list]
3

3

In [74]:
rho_num = np.zeros_like(rho_num_list[0])

# Loop over the matrices and sum them element-wise
for matrix in rho_num_list:
    rho_num = np.add(rho_num, matrix)

rho_num   

array([[0.625522+0.00000000e+00j, 0.176975-1.48783164e-03j,
        0.125324+8.31384388e-05j],
       [0.176975+1.48783164e-03j, 0.249492+3.32524347e-18j,
        0.176589-5.35203700e-04j],
       [0.125324-8.31384388e-05j, 0.176589+5.35203700e-04j,
        0.124986+2.32342109e-18j]])

In [75]:
rho # printing the theoretical density matrix and below the numerically reconstructed one

Matrix([
[            0.625, 0.176776695296637,             0.125],
[0.176776695296637,              0.25, 0.176776695296637],
[            0.125, 0.176776695296637,             0.125]])

In [76]:
np.matrix(rho_num)  

matrix([[0.625522+0.00000000e+00j, 0.176975-1.48783164e-03j,
         0.125324+8.31384388e-05j],
        [0.176975+1.48783164e-03j, 0.249492+3.32524347e-18j,
         0.176589-5.35203700e-04j],
        [0.125324-8.31384388e-05j, 0.176589+5.35203700e-04j,
         0.124986+2.32342109e-18j]])

In [77]:
rho 
del_rho = rho_num-rho
del_rho

Matrix([
[                        0.000521999999999911,   0.000198304703363156 - 0.00148783164370175*I,   0.000323999999999991 + 8.31384387632583e-5*I],
[0.000198304703363156 + 0.00148783164370175*I, -0.000507999999999953 + 3.32524347327919e-18*I, -0.000187695296636897 - 0.000535203699538781*I],
[0.000323999999999991 - 8.31384387632583e-5*I, -0.000187695296636897 + 0.000535203699538781*I,  -1.40000000000418e-5 + 2.32342109202537e-18*I]])

In [78]:
# evects = del_rho.eigenvects()
eigensystem = del_rho.eigenvects()

#separating eigenvalues of del rho
evects_0 = []
evals_0 = []

for i in range(3):
    evals_0.append(eigensystem[i][0])
    evects_0.append(eigensystem[i][-1])

evals = [sym.re(i) for i in evals_0]
norm_del_rho = max([abs(i) for i in evals])     # picking out the largest evalue

evals

[0.00160836568560192, -0.00181005069982268, 0.000201685014220678]

In [79]:
#eigenvalues of rho_num
eigensystem_num = sym.Matrix(rho_num).eigenvects()
evects_num = []
evals_num = []

for i in range(3):
    evals_num.append(eigensystem_num[i][0])
    evects_num.append(eigensystem_num[i][-1])

# evals = [sym.re(i) for i in evals_0]
evals_num = [sym.re(i) for i in evals_num]

zero_eval_rho_num = min(evals_num)                          # picking the minimum evalue 
zero_eval_index = evals_num.index(zero_eval_rho_num)        # finding index so we can pick the corresponding evector
perp_evect_rho_num = evects_num[zero_eval_index]

c00 = perp_evect_rho_num[0][0]
c01 = perp_evect_rho_num[0][1]
c11 = perp_evect_rho_num[0][2]         # the coefficients of the xi vector

perp_evect_rho_num



[Matrix([
 [0.000124543041773958 + 0.00247405556218167*I],
 [   0.577640316077404 - 0.00661727457882405*I],
 [  -0.816248996746978 + 0.00439497062585415*I]])]

In [80]:
Quadratic_Form = lambda a,b,c : [(-b+ ((b**2-4*a*c)**(.5)))/(2*a), (-b- ((b**2-4*a*c)**.5))/(2*a)]

In [81]:
[sym.simplify(Quadratic_Form(c00,c01,c11)[0]), sym.simplify(Quadratic_Form(c00,c01,c11)[1])]

[-4.5278189612416 + (1 + 0.00128948078638782*I)**0.5*(5.86534019766437 - 116.515361544297*I) + 116.511631387323*I,
 -4.5278189612416 + (-5.86534019766437 + 116.515361544297*I)*(1 + 0.00128948078638782*I)**0.5 + 116.511631387323*I]